# Machine Learning Algorithm For Identifying Exoplanets in Kepler Data
## Status: In progress

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

import csv
import sys
sys.path.append('../astroML-coral/Tools')
sys.path.append('../astroML-coral/Data')
import datascrub as scrub
import mldataconfig as dtc
import pprint

# Import Data Sets

Note: file paths will need to be adjusted based on individual user

In [2]:
#scrub.exofinder('testsh17.txt', 'bigtest.csv')
#dtc.datasplit('bigtest.csv')

traindat = pd.read_csv("training.csv")
train = traindat.to_numpy()
testdat = pd.read_csv("test.csv")
test = testdat.to_numpy()

In [3]:
train.shape
test.shape

(9244, 4372)

# Setup

Flux values set as array of inputs for the model, the binary yes/no of is there an exoplanet is set as the output array
This is also done on the test data to prep for later 

In [4]:
x = train[:,2:]
y = train[:,1]
test_x = test[:,1:]
test_y = test[:,0]

print(f"Train dataset shape: {x.shape}")
print(f"Train-res dataset shape: {y.shape}")

Train dataset shape: (99998, 1)
Train-res dataset shape: (99998,)


# Machine Learning

A basic keras sequential model was trained on the provided dataset. The model consisted of an initialization layer, two layers with different activation equations, and an output layer. While the initilization and calculation layers had a rather arbitrary number of nodes that was experimentally deemed "good enough", the output layer has only two possible outcomes, yes/no does the system contain an exoplanet. 

Finally, the model is compiled with Tensorflow's Adam optimizer and a loss function selected for its good performance in this scenario.

In [5]:
model = Sequential()
model.add(Dense(64))
model.add(Dense(64, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(x, y, epochs=20)

Epoch 1/20
159/159 [==============================] - 1s 1ms/step - loss: 0.4270 - accuracy: 0.8591
Epoch 2/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0505 - accuracy: 0.9927
Epoch 3/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0378 - accuracy: 0.9937
Epoch 4/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0365 - accuracy: 0.9943
Epoch 5/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0422 - accuracy: 0.9926
Epoch 6/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0405 - accuracy: 0.9930
Epoch 7/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0453 - accuracy: 0.9916
Epoch 8/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0338 - accuracy: 0.9942
Epoch 9/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0387 - accuracy: 0.9936
Epoch 10/20
159/159 [==============================] - 0s 1ms/step - loss: 0.0417 - accuracy: 0.9929

# Analysis

The machine learning algorithm hits its peak accuracy relatively quickly, with every run I've done hitting ~99% within the first three epochs. Running this on the test data set results in the same 99% accuracy, showing that the data is being properly processed and not over or under-sampled.

In [6]:
test_loss, test_acc = model.evaluate(test_x,  test_y, verbose=1)

print('\nTest accuracy:', test_acc)

18/18 [==============================] - 0s 938us/step - loss: 0.0475 - accuracy: 0.9912

Test accuracy: 0.9912280440330505


In [7]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_x)
predictions[0]

array([0.9947019 , 0.00529802], dtype=float32)